In [3]:
import pygame
import random
import math

# initialize pygame
pygame.init()
width, height = 600, 500
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Shadow Dodge: Space Adventure")

# giving colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 100, 255)
RED = (255, 50, 50)
GREEN = (50, 255, 50)
YELLOW = (255, 255, 100)
PURPLE = (150, 50, 200)
ORANGE = (255, 150, 50)
DARK_BLUE = (5, 5, 30)
DARK_SPACE = (2, 2, 15)

# fonts for intstruction and score
font_small = pygame.font.SysFont('Arial', 20, bold=True)
font_medium = pygame.font.SysFont('Arial', 30, bold=True)
font_large = pygame.font.SysFont('Arial', 50, bold=True)

# player that is the spaceship properties 
player_size = 35
player_x = width // 2 - player_size // 2
player_y = height - 60
player_speed = 8
player_lives = 3
shield_active = False
shield_timer = 0
speed_boost_active = False
speed_boost_timer = 0

# shadow types (now space debris)
SHADOW_TYPES = [
    {"width": 60, "height": 20, "speed": 4, "color": (80, 80, 90), "score": 1, "outline": (150, 150, 150)},
    {"width": 40, "height": 40, "speed": 6, "color": (150, 70, 70), "score": 2, "outline": (200, 100, 100)},
    {"width": 80, "height": 30, "speed": 3, "color": (100, 50, 150), "score": 3, "outline": (150, 80, 200)},
    {"width": 30, "height": 60, "speed": 5, "color": (220, 150, 50), "score": 2, "outline": (255, 180, 80)}
]
shadows = []

# power-ups (now space power-ups) power ups for boosting player life like immunity and all
POWER_UPS = [
    {"type": "shield", "color": BLUE, "outline": WHITE, "duration": 5000, "size": 20},
    {"type": "speed", "color": GREEN, "outline": BLACK, "duration": 3000, "size": 20},
    {"type": "life", "color": RED, "outline": BLACK, "size": 20}
]
powerups = []
powerup_spawn_timer = 0

# game variables
clock = pygame.time.Clock()
score = 0
high_score = 0
game_time = 0
particles = []
difficulty = 1
spawn_rate = 30
stars = []
nebulas = []

def init_stars():
    """Initialize starfield background with more stars and nebulas"""
    global stars, nebulas
    stars = []
    nebulas = []
    
    # creating 500 stars
    for _ in range(500):
        stars.append([
            random.randint(0, width),  # x
            random.randint(0, height),  # y
            random.uniform(0.1, 1.2),  # size (some bigger  stars )
            random.randint(50, 255),  # brightness of stars
            random.uniform(0.01, 0.1)  # twinkle speed of stars
        ])
    
    # creating 4 nebula clouds
    for _ in range(4):
        nebulas.append({
            "x": random.randint(0, width),
            "y": random.randint(0, height//2),
            "size": random.randint(100, 300),
            "color": (
                random.randint(30, 100),
                random.randint(10, 50),
                random.randint(50, 100)
            ),
            "speed": random.uniform(0.1, 0.3)
        })

def draw_background():
    """Draw enhanced space background with nebulas and stars"""
    screen.fill(DARK_SPACE)  # Darker space color
    
    # draw nebula clouds
    for nebula in nebulas:
        # Animate nebula movement slightly
        nebula["x"] += math.sin(game_time/5000) * nebula["speed"]
        nebula["y"] += math.cos(game_time/7000) * nebula["speed"]
        
        # create gradient nebula effect
        for i in range(3, 0, -1):
            size = nebula["size"] * i/3
            alpha = 30 * i
            color = (*nebula["color"], alpha)
            s = pygame.Surface((size*2, size*2), pygame.SRCALPHA)
            pygame.draw.circle(s, color, (size, size), size)
            screen.blit(s, (int(nebula["x"] - size), int(nebula["y"] - size)))
    
    # draw distant galaxies (small fuzzy circles)
    for i in range(3):
        gal_x = width * 0.2 + math.sin(game_time/8000 + i) * width * 0.1
        gal_y = height * 0.1 + math.cos(game_time/9000 + i*1.5) * height * 0.1
        gal_size = 80 + math.sin(game_time/3000 + i) * 20
        gal_color = (50 + i*20, 30 + i*10, 80 + i*30, 50)
        s = pygame.Surface((gal_size*2, gal_size*2), pygame.SRCALPHA)
        pygame.draw.circle(s, gal_color, (gal_size, gal_size), gal_size)
        screen.blit(s, (int(gal_x - gal_size), int(gal_y - gal_size)))
    
    # draw stars with enhanced twinkle effect
    for star in stars:
        # More pronounced twinkle effect
        twinkle = math.sin(game_time * star[4]) * 40
        brightness = max(30, min(255, star[3] + twinkle))
        
        # color variation based on brightness
        if brightness > 200:  # Very bright stars are white
            color = (brightness, brightness, brightness)
        elif brightness > 150:  # Medium stars are blue-white
            color = (brightness, brightness, brightness + 30)
        else:  # Dim stars are blue
            color = (brightness//2, brightness//2, brightness)
        
        # draw star
        pygame.draw.circle(screen, color, (int(star[0]), int(star[1])), star[2])
        
        # add lens flare effect to some stars
        if star[2] > 0.8 and random.random() < 0.01:
            flare_size = star[2] * 3
            alpha = min(100, brightness)
            s = pygame.Surface((flare_size*2, flare_size*2), pygame.SRCALPHA)
            pygame.draw.circle(s, (*color[:3], alpha//2), (flare_size, flare_size), flare_size)
            screen.blit(s, (int(star[0] - flare_size), int(star[1] - flare_size)))

def draw_player():
    """Draw player as a spaceship"""
    # Draw spaceship body
    pygame.draw.polygon(screen, BLUE, [
        (player_x + player_size//2, player_y),
        (player_x, player_y + player_size),
        (player_x + player_size, player_y + player_size)
    ])
    
    # draw spaceship cockpit
    pygame.draw.circle(screen, WHITE, 
                      (player_x + player_size//2, player_y + player_size//3), 
                      player_size//4)
    
    # draw engine glow when moving
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] or keys[pygame.K_RIGHT]:
        pygame.draw.polygon(screen, ORANGE, [
            (player_x + player_size//3, player_y + player_size),
            (player_x + player_size//2, player_y + player_size + 10),
            (player_x + player_size*2//3, player_y + player_size)
        ])
    
    # draw shield effect if active
    if shield_active:
        shield_size = player_size + 15 + math.sin(game_time/100) * 5
        alpha = 100 + int(155 * (shield_timer - pygame.time.get_ticks()) / 5000)
        s = pygame.Surface((shield_size*2, shield_size*2), pygame.SRCALPHA)
        pygame.draw.circle(s, (0, 150, 255, alpha), 
                          (shield_size, shield_size), shield_size, 3)
        screen.blit(s, (player_x - shield_size//2 + player_size//2 - 2, 
                       player_y - shield_size//2 + player_size//2 - 2))

def show_instructions():
    """Show game instructions screen"""
    instructions = [
        "SPACE DEBRIS DODGE",
        "",
        "Avoid falling space debris",
        "Use LEFT and RIGHT arrow keys to move",
        "",
        "Power-ups:",
        "Blue - Shield (temporary protection)",
        "Green - Speed Boost (faster movement)",
        "Red - Extra Life",
        "",
        "Press any key to start!"
    ]
    
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            if event.type == pygame.KEYDOWN:
                waiting = False
        
        draw_background()
        
        # draw title
        title = font_large.render(instructions[0], True, YELLOW)
        screen.blit(title, (width//2 - title.get_width()//2, 50))
        
        # draw instructions
        for i, line in enumerate(instructions[1:]):
            if line:
                text = font_small.render(line, True, WHITE)
                screen.blit(text, (width//2 - text.get_width()//2, 150 + i * 30))
        
        pygame.display.update()
        clock.tick(60)
    
    return True

def create_shadow():
    """Create a new space debris object"""
    shadow_type = random.choice(SHADOW_TYPES)
    shadow = {
        "x": random.randint(0, width - shadow_type["width"]),
        "y": -shadow_type["height"],
        "width": shadow_type["width"],
        "height": shadow_type["height"],
        "speed": shadow_type["speed"] * difficulty,
        "color": shadow_type["color"],
        "outline": shadow_type["outline"],
        "score": shadow_type["score"]
    }
    shadows.append(shadow)

def draw_shadow(shadow):
    """Draw a space debris object"""
    # Draw main body
    pygame.draw.rect(screen, shadow["color"], 
                    (shadow["x"], shadow["y"], shadow["width"], shadow["height"]))
    # Draw outline
    pygame.draw.rect(screen, shadow["outline"], 
                    (shadow["x"], shadow["y"], shadow["width"], shadow["height"]), 2)
    # Add some details to make it look like space debris
    if shadow["width"] > 40 and shadow["height"] > 30:
        pygame.draw.line(screen, shadow["outline"],
                         (shadow["x"] + 10, shadow["y"] + 15),
                         (shadow["x"] + shadow["width"] - 10, shadow["y"] + 15), 2)

def create_powerup():
    """Create a new power-up"""
    powerup_type = random.choice(POWER_UPS)
    powerup = {
        "x": random.randint(20, width - 40),
        "y": -30,
        "size": powerup_type["size"],
        "type": powerup_type["type"],
        "color": powerup_type["color"],
        "outline": powerup_type["outline"],
        "speed": 3,
        "duration": powerup_type.get("duration", 0)
    }
    powerups.append(powerup)

def draw_powerup(powerup):
    """Draw a power-up"""
    # Draw main shape
    pygame.draw.circle(screen, powerup["color"], 
                      (int(powerup["x"] + powerup["size"]//2), 
                      int(powerup["y"] + powerup["size"]//2)), 
                      powerup["size"]//2)
    # draw outline
    pygame.draw.circle(screen, powerup["outline"], 
                      (int(powerup["x"] + powerup["size"]//2), 
                       int(powerup["y"] + powerup["size"]//2)), 
                      powerup["size"]//2, 2)
    
    # draw symbol based on type
    if powerup["type"] == "shield":
        # Draw shield symbol
        pygame.draw.circle(screen, powerup["outline"], 
                          (int(powerup["x"] + powerup["size"]//2), 
                           int(powerup["y"] + powerup["size"]//2)), 
                          powerup["size"]//3, 2)
    elif powerup["type"] == "speed":
        # draw lightning bolt
        points = [
            (powerup["x"] + powerup["size"]//2, powerup["y"] + powerup["size"]//3),
            (powerup["x"] + powerup["size"]*2//3, powerup["y"] + powerup["size"]//2),
            (powerup["x"] + powerup["size"]//3, powerup["y"] + powerup["size"]//2),
            (powerup["x"] + powerup["size"]//2, powerup["y"] + powerup["size"]*2//3)
        ]
        pygame.draw.lines(screen, powerup["outline"], False, points, 3)
    elif powerup["type"] == "life":
        # draw heart
        center_x = powerup["x"] + powerup["size"]//2
        center_y = powerup["y"] + powerup["size"]//2
        pygame.draw.polygon(screen, powerup["outline"], [
            (center_x, center_y - powerup["size"]//4),
            (center_x + powerup["size"]//4, center_y),
            (center_x, center_y + powerup["size"]//4),
            (center_x - powerup["size"]//4, center_y)
        ])

def create_particles(x, y, color):
    """Create explosion particles"""
    for _ in range(20):
        angle = random.uniform(0, math.pi * 2)
        speed = random.uniform(1, 5)
        particles.append({
            "x": x,
            "y": y,
            "speed_x": math.cos(angle) * speed,
            "speed_y": math.sin(angle) * speed,
            "color": color,
            "size": random.randint(2, 5),
            "life": random.randint(20, 40)
        })

def draw_particles():
    """Draw all particles"""
    for particle in particles:
        alpha = min(255, particle["life"] * 6)
        color = (*particle["color"], alpha) if len(particle["color"]) == 3 else particle["color"]
        if len(color) == 4:
            s = pygame.Surface((particle["size"]*2, particle["size"]*2), pygame.SRCALPHA)
            pygame.draw.circle(s, color, (particle["size"], particle["size"]), particle["size"])
            screen.blit(s, (int(particle["x"] - particle["size"]), int(particle["y"] - particle["size"])))
        else:
            pygame.draw.circle(screen, color, 
                              (int(particle["x"]), int(particle["y"])), 
                              particle["size"])

def draw_ui():
    """Draw user interface elements"""
    # draw score
    score_text = font_medium.render(f"Score: {score}", True, WHITE)
    screen.blit(score_text, (20, 20))
    
    # draw high score
    high_score_text = font_small.render(f"High Score: {high_score}", True, WHITE)
    screen.blit(high_score_text, (20, 60))
    
    # draw lives
    for i in range(player_lives):
        pygame.draw.polygon(screen, RED, [
            (width - 40 - i * 30, 30),
            (width - 60 - i * 30, 50),
            (width - 40 - i * 30, 70),
            (width - 20 - i * 30, 50)
        ])
    
    # draw power-up indicators
    if shield_active:
        time_left = max(0, (shield_timer - pygame.time.get_ticks()) / 1000)
        shield_text = font_small.render(f"Shield: {time_left:.1f}s", True, BLUE)
        screen.blit(shield_text, (width - 150, height - 30))
    
    if speed_boost_active:
        time_left = max(0, (speed_boost_timer - pygame.time.get_ticks()) / 1000)
        speed_text = font_small.render(f"Speed: {time_left:.1f}s", True, GREEN)
        screen.blit(speed_text, (width - 150, height - 60))

def game_over_screen():
    """Show game over screen and return player choice"""
    options = ["Restart", "Quit"]
    selected = 0
    
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return "quit"
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    selected = (selected - 1) % len(options)
                elif event.key == pygame.K_RIGHT:
                    selected = (selected + 1) % len(options)
                elif event.key == pygame.K_RETURN:
                    return options[selected].lower()
        
        draw_background()
        
        # Draw game over text
        game_over = font_large.render("GAME OVER", True, RED)
        screen.blit(game_over, (width//2 - game_over.get_width()//2, 100))
        
        # Draw final score
        score_text = font_medium.render(f"Final Score: {score}", True, WHITE)
        screen.blit(score_text, (width//2 - score_text.get_width()//2, 180))
        
        # Draw high score
        high_score_text = font_medium.render(f"High Score: {high_score}", True, YELLOW)
        screen.blit(high_score_text, (width//2 - high_score_text.get_width()//2, 230))
        
        # Draw options
        for i, option in enumerate(options):
            color = YELLOW if i == selected else WHITE
            text = font_medium.render(option, True, color)
            x_pos = width//2 - 100 + i * 200
            screen.blit(text, (x_pos - text.get_width()//2, 300))
        
        pygame.display.update()
        clock.tick(60)
    
    return "quit"

def reset_game():
    """Reset game variables for a new game"""
    global player_x, player_y, player_lives, score, shadows, powerups, particles
    global shield_active, speed_boost_active
    
    player_x = width // 2 - player_size // 2
    player_y = height - 60
    player_lives = 3
    score = 0
    shadows = []
    powerups = []
    particles = []
    shield_active = False
    speed_boost_active = False

def main():
    global player_x, player_y, player_lives, score, high_score
    global game_time, difficulty, spawn_rate, shield_active, speed_boost_active
    
    init_stars()
    if not show_instructions():
        return
    
    running = True
    start_time = pygame.time.get_ticks()
    powerup_spawn_timer = pygame.time.get_ticks()
    
    while running:
        current_time = pygame.time.get_ticks()
        game_time = current_time - start_time
        clock.tick(60)
        
        # increase difficulty
        difficulty = 1 + min(game_time // 30000, 4)
        spawn_rate = max(10, 30 - (game_time // 5000))
        
        # event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        # player movement
        current_speed = player_speed * (1.5 if speed_boost_active else 1)
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and player_x > 0:
            player_x -= current_speed
        if keys[pygame.K_RIGHT] and player_x < width - player_size:
            player_x += current_speed
        
        # spawn shadows (space debris)
        if random.randint(1, spawn_rate) == 1:
            create_shadow()
        
        # spawn power-ups
        if current_time - powerup_spawn_timer > random.randint(10000, 15000):
            create_powerup()
            powerup_spawn_timer = current_time
        
        # draw everything
        draw_background()
        
        # update and draw shadows
        for shadow in shadows[:]:
            shadow["y"] += shadow["speed"]
            draw_shadow(shadow)
            
            # collision detection
            if (not shield_active and 
                shadow["y"] + shadow["height"] >= player_y and
                shadow["x"] < player_x + player_size and
                shadow["x"] + shadow["width"] > player_x):
                
                create_particles(shadow["x"] + shadow["width"]//2, 
                                shadow["y"] + shadow["height"]//2, 
                                shadow["color"])
                shadows.remove(shadow)
                player_lives -= 1
                
                if player_lives <= 0:
                    running = False
            
            # remove off-screen shadows
            if shadow["y"] > height:
                shadows.remove(shadow)
                score += shadow["score"]
        
        # update and draw power-ups
        for powerup in powerups[:]:
            powerup["y"] += powerup["speed"]
            draw_powerup(powerup)
            
            # collection check
            if (powerup["x"] < player_x + player_size and
                powerup["x"] + powerup["size"] > player_x and
                powerup["y"] < player_y + player_size and
                powerup["y"] + powerup["size"] > player_y):
                
                if powerup["type"] == "shield":
                    shield_active = True
                    shield_timer = current_time + powerup["duration"]
                elif powerup["type"] == "speed":
                    speed_boost_active = True
                    speed_boost_timer = current_time + powerup["duration"]
                elif powerup["type"] == "life":
                    player_lives = min(5, player_lives + 1)
                
                create_particles(powerup["x"] + powerup["size"]//2, 
                                powerup["y"] + powerup["size"]//2, 
                                powerup["color"])
                powerups.remove(powerup)
            
            # remove off-screen
            if powerup["y"] > height:
                powerups.remove(powerup)
        
        # update power-ups
        if shield_active and current_time > shield_timer:
            shield_active = False
        if speed_boost_active and current_time > speed_boost_timer:
            speed_boost_active = False
        
        # update particles
        for particle in particles[:]:
            particle["x"] += particle["speed_x"]
            particle["y"] += particle["speed_y"]
            particle["life"] -= 1
            if particle["life"] <= 0:
                particles.remove(particle)
        
        # draw particles
        draw_particles()
        
        # draw player
        draw_player()
        
        # draw UI
        draw_ui()
        
        pygame.display.update()
    
    # game over
    high_score = max(score, high_score)
    result = game_over_screen()
    if result == "restart":
        reset_game()
        main()

if __name__ == "__main__":
    main()
    pygame.quit()